<a href="https://colab.research.google.com/github/pixi2102/iad-intro-ds/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22hw02_pandas_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2 (5 баллов).

*Все задания ниже имеют равный вес (5/10)*

Код для импорта мы написали за вас (не благодарите, нам не трудно). Дальше код будете писать вы.

[Тут](https://habr.com/ru/companies/ruvds/articles/494720/) шпора по pandas. За основу домашнего задания взят ноутбук [отсюда](https://rutube.ru/video/f884aa6ed5f94120b7304506042fe5bb/) (не подглядывайте!).

In [3]:
import pandas as pd
import numpy as np

#### Описание данных

Автор д/з - плохой человек, который не стал переводить описание с мотивировкой, что весь DS на английском. Так что описание полей будет на английском:

1. Account ID
- Description: A unique identifier for each social media account in the dataset.
- Type: Integer
- Example: 1, 2, 3, …
2. Username
- Description: The username or handle of the social media account.
- Type: String
- Example: john_doe, tech_guru_22, fitness_freak
3. Platform
- Description: The social media platform the account is using (Instagram, Twitter, Facebook, TikTok, LinkedIn).
- Type: Categorical (String)
- Example: Instagram, Twitter, Facebook, TikTok, LinkedIn
4. Follower Count
- Description: The total number of followers the account has.
- Type: Integer
- Example: 1500, 245000, 78000
5. Posts Per Week
- Description: The average number of posts the account creates per week.
- Type: Integer
- Example: 3, 5, 7
6. Engagement Rate
- Description: The percentage of interactions (likes, comments, shares) relative to the follower count. This is a measure of how engaging the content is.
- Type: Float
- Range: 0.01 to 0.15
- Example: 0.045 (4.5% engagement rate)
7. Ad Spend (USD)
- Description: The monthly amount spent on advertising or promoting posts.
- Type: Float
- Example: 150.75, 850.00, 300.50
8. Conversion Rate
- Description: The percentage of users who take a desired action (e.g., clicking a link, signing up, etc.) after interacting with an ad.
- Type: Float
- Range: 0.01 to 0.05 (1% to 5% conversion rate)
- Example: 0.025 (2.5% conversion rate)
9. Campaign Reach
- Description: The total number of unique users reached by the user’s campaigns in a given month.
- Type: Integer
- Example: 5000, 20000, 15000

#### Задание 0

Подгрузите данные. Да-да, за чтение таблицы баллов не будет))

**Hint**: [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [5]:
df = pd.read_csv('data (8).csv')# ^⨀ᴥ⨀^

#### Задание 1

Колонка `Platform` содержит название различных платформ. Давайте представим, что в них есть некоторое отношение порядка. Закодируйте каждую платформу целым числом (от 0 до N) и положите этот "код" в новую колонку `Platform_Code`. Теперь вычислите корреляцию Спирмена между всеми парами колонок в датасете (результатом будет таблица корреляций). В качестве ответа выведите значение корреляции `Platform_Code` с `Engagement Rate`. Можете после вывода числа еще коротко написать, что оно означает (нет, это не оценивается).

**Hint**: [pd.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html), [pd.DataFrame.select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [pd.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

In [6]:
df['Platform_Code'], unique = pd.factorize(df['Platform'])
df1 = df.select_dtypes(include=['number'])
correlation_matrix = df1.corr(method='spearman')
print(correlation_matrix.loc['Platform_Code', 'Engagement Rate'])

0.03138169529349812


#### Задание 2

Теперь посмотрите на столбец `Follower Count`. В нем какие-то числа. Иногда бывает полезно провести дискретизацию такого признака. Разбейте все значения в столбце на 4 группы: "Low", "Medium", "High", "Very High". Каждая группа включает в себя новые 25% данных. То есть, Low включает в себя 25% самых маленьких значений признака и так далее. Положите значения "Low", "Medium", "High" или "Very High" для каждого сэмпла датасета в новую колонку `Follower_Bin`. Теперь посчитайте среднее значение `Engagement Rate` для каждой категории из `Follower_Bin`. В качестве ответа выведите значение для категории "High".

**Hint**: [pd.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html), [pd.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), [pd.DataFrame.mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)

In [7]:
df['Follower_Bin']=pd.qcut(df['Follower Count'], 4, labels=['Low','Medium', 'High','Very High'])
print(df.groupby('Follower_Bin')['Engagement Rate'].mean()['High'])

0.08655032


<ipython-input-7-e2e6a664b0ce>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Follower_Bin')['Engagement Rate'].mean()['High'])


#### Задание 3

Иногда бывает полезно превратить широкую таблицу в длинную (например, для визуализаций сразу нескольких признаков на одной картинке). Да, звучит странно, но именно этим вы сейчас и займетесь. Сделайте новый датафрейм `melted_df`, в который вы поместите каждый сэмпл датасета 6 раз: по одному разу на значение из 'Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate' и 'Campaign Reach'. То есть, вы берете сэмпл из датасета (строку) и превращаете ее в 6 отдельных строк. Каждая отдельная строка в столбце `Metric` имеет имя из предложенного списка 5 признаков, а в столбце `Value` - значение данного сэмпла по этому признаку. Значение `Platform` повторяется в этих 6 строках.

Иначе говоря,

```json
{
    "Account ID": 1,
    "Username": "harrislisa",
    "Platform": "TikTok",
    "Follower Count": 54217,
    "Posts Per Week": 3,
    "Engagement Rate": 0.0986,
    "Ad Spend (USD)": 538.1,
    "Conversion Rate": 0.049,
    "Campaign Reach": 1308,
    "Platform_Code": 0,
    "Follower_Bin": "Low"
}
```

превращается в

```json
{
    "Platform": "TikTok",
    "Metric": "Follower Count",
    "Value": 54217,
},
{
    "Platform": "TikTok",
    "Metric": "Posts Per Week",
    "Value": 3,
}, ...
```

Для каждого уникальной пары значений (`Platform`, `Metric`) посчитайте моду среди всех значений `Value` для этой пары, результат сделайте списком и оставьте только наибольшее. В качестве ответа выведите сумму полученных мод (сумму всех значений в столбце `Value` уже после вычисления мод). Иначе говоря, выведите сумму всех мод значений для всех уникальных пар (`Platform`, `Metric`).

**Hint**: [pd.melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html), [pd.DataFrame.mode](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html), [pd.DataFrameGroupBy.agg](https://pandas.pydata.org/docs/dev/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html)

In [8]:
melted_df=pd.melt(df,id_vars=['Platform'],
                  value_vars=['Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate','Campaign Reach'],
                  var_name='Metric',
                  value_name='Value')# (づ๑•ᴗ•๑)づ♡
mode_values = melted_df.groupby(['Platform', 'Metric'])['Value'].agg(lambda x: x.mode().max()).reset_index()
print(mode_values['Value'].sum())

3100285.4716


#### Задание 4

А теперь хочется посмотреть на самые популярные аккаунты на разных платформах. Для каждой платформы отсортируйте датафрейм по убыванию количества подписчиков (`Follower Count`) - да, без циклов, сразу для всех платформ сделать сортировку, а затем оставьте только первые три записи для каждой платформы - это и будут три самых популярных аккаунта для каждой платформы. В качестве ответа выведите саму таблицу и минимальное значение `Follower Count` в ней.

**Hint**: к *groupby* можно применять функции - это эквивалентно применению функции к каждой "группе" внутри groupby-объекта. Читайте [про применение apply к датафрейму после groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

In [9]:
df_updated=df.groupby('Platform').apply(lambda x: x.sort_values(by='Follower Count', ascending=False).head(3)).reset_index(drop=True)# ε(´סּ︵סּ`)з
print(df_updated)
print(df_updated['Follower Count'].min())

    Account ID         Username   Platform  Follower Count  Posts Per Week  \
0         2404           eric65   Facebook          999982               6   
1         7351     patricknoble   Facebook          997915               3   
2         1690      chavezjason   Facebook          997512               7   
3         8686  alexandersamuel  Instagram          999726               3   
4         3966         lrodgers  Instagram          999351               1   
5         2190           jbrown  Instagram          997844               5   
6         3040          toneill   LinkedIn          999055               4   
7         6360    andrewgregory   LinkedIn          998968               7   
8         2160     ashleycooper   LinkedIn          998925               6   
9         5839     edwardthomas     TikTok          999739               7   
10        4235    andradewesley     TikTok          999234               5   
11        2576     williamwyatt     TikTok          998623      

<ipython-input-9-71a790d5d725>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_updated=df.groupby('Platform').apply(lambda x: x.sort_values(by='Follower Count', ascending=False).head(3)).reset_index(drop=True)# ε(´סּ︵סּ`)з


#### Задание 5

Хочется посчитать какую-то метрику. Мы хотим посмотреть, на отношение разности суммы подписчиков аккаунтов с высокой и низкой конверсией к суммарному охвату рекламы на каждой платформе. То есть, мы делим аккаунты на две группы: высокая и низка конверсия. Затем мы смотрим на то, на сколько сильно влияние аккаунтов с высокой конверсией по сравнению с аккаунтами с низкой конверсией.

Давайте определим *Conversion Influence* следущим образом:

$$Conversion Influence = \frac{Total Follower\ Count (High) - Total Follower\ Count (Low)}{Total Campaign Reach (High)+Total Campaign Reach (Low)}$$

Считать эту метрику мы будет для каждой `Platform`. В этой формуле High - это значения всех сэмплов датасета, в которых `Conversion Rate` больше медианы, а `Low` - не более медианы. `Total Feature` - это суммарное количество значений `Feature` либо по `High` сэмплам, либо по `Low`.

Чтобы постоянно не пересчитывать, где High. где Low, сделайте новую колонку в датасете `Conversion_Category`. Положите в нее для каждой строки либо High, либо Low.

Выведите платформу с самым большим `Conversion Influence`.

**Hint**: данное задание не про *groupby*, а скорее про [pd.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html). Сделайте сводную таблицу, по которой уже можно посчитать суммы, а затем подставить их в формулы.

In [10]:
median = df['Conversion Rate'].median()
df['Conversion_Category'] = df['Conversion Rate'].apply(lambda x: 'High' if x > median else 'Low')

pivot_table = pd.pivot_table(df,
                             values=['Follower Count', 'Campaign Reach'],
                             index='Platform',
                             columns='Conversion_Category',
                             aggfunc='sum',
                             fill_value=0)
pivot_table['ConversionInfluence'] = (
    (pivot_table[('Follower Count', 'High')] - pivot_table[('Follower Count', 'Low')]) /
    (pivot_table[('Campaign Reach', 'High')] + pivot_table[('Campaign Reach', 'Low')])
)

print(pivot_table['ConversionInfluence'].idxmax())
# (︶ω︶)

Twitter


#### Задание 6

Мы знаем, что вам понравилось считать метрики по формуле. Давайте закрепим этот успех. Теперь для каждой платформы посчитаем, на сколько эффективна реклама в разрезе трех последовательных записей в датасете.

Для каждой платформы отсортируйте записи в порядке убывания `Posts Per Week`. Будто бы аккаунты, которые постят чаще, используют более "активные" стратегии по рекламе. Теперь посчитайте *скользущие суммы с окном 3* по `Campaign Reach` и `Ad Spend (USD)`. Скользящая сумма с окном N - это вы идете по массиву, берете все последовательные тройки записей и суммируете их. Для первых двух записей троек не найдется. Для них скользящее среднее - NaN, что нам не помешает.

Теперь для каждого окна посчитайте

$$Rolling Efficiency Ratio = \frac{Rolling Sum of Campaign Reach}{Rolling Sum of Ad Spend}$$

По сути, для каждого окна вы посчитаете сколько пользователе привлеклось за один доллар, потреченный на рекламу, в данном окне. Понятно, что значений будет столько, сколько окон. Нам интересно максимально значение такой эффективности для каждой платформы.

В качестве ответа выведите название платформы с наибольшей максимальной эффективность и наименьшей (два названия, не одно, не три, ровно два).

**Hint**: окна можно делать через [pd.DataFrame.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html).

In [11]:
df_sorted = df.sort_values(by=['Platform', 'Posts Per Week'], ascending=False)

df_sorted['RollingSumReach'] = df_sorted.groupby('Platform')['Campaign Reach'].rolling(window=3).sum().reset_index(drop=True)
df_sorted['RollingSumSpend'] = df_sorted.groupby('Platform')['Ad Spend (USD)'].rolling(window=3).sum().reset_index(drop=True)

df_sorted['RollingEfficiencyRatio'] = df_sorted['RollingSumReach'] / df_sorted['RollingSumSpend']

efficiency = df_sorted.groupby('Platform')['RollingEfficiencyRatio'].max().reset_index()

print(efficiency.loc[efficiency['RollingEfficiencyRatio'].idxmax(), 'Platform'])
print(efficiency.loc[efficiency['RollingEfficiencyRatio'].idxmin(), 'Platform'])


Twitter
Facebook


#### Задание 7

Это еще не все прекрасные функции pandas, которые мы хотим вам показать. Теперь вы посчитаете, сколько аккаунтов на каждой платформе одновременно лучшие по `Engagement Rate` и `Conversion Rate`.

Сделайте два отдельных суб-сета. В одном оставьте для каждой платфмормы один топовый аккаунт по `Engagement Rate`, в другом - по `Conversion Rate`. Соедините эти два подмножества по столбцу `Platform` так, что в одно строке есть описание сразу двух аккаунтов-лидеров. Теперь посмотрите равны ли имена аккаунтов в одной строке. Выведите количество строк, в которых названия аккаунтов совпадают.

In [22]:
df1=df.loc[df.groupby('Platform')['Engagement Rate'].idxmax()]
df2=df.loc[df.groupby('Platform')['Conversion Rate'].idxmax()]
df1 = df1.rename(columns={'Username': 'Username_1'})
df2 = df2.rename(columns={'Username': 'Username_2'})
df_united=pd.merge(df1,df2, on='Platform')
df_united['Same Acc']=(df_united['Username_1']==df_united['Username_2'])
print(df_united['Same Acc'].sum())# ( ͡° ͜ʖ ͡°)

0


#### Задание 8

Давайте теперь что-то попроще сделаем. Например, посчитаем отношение суммарного количества подписчиков на аккаунтах с высокой конверсией к такой же сумме в аккаунтах с низкой конверсией (очевидно, для каждой платформы). По сути, мы просто хотим получить число, которое характеризует, на сколько сильно аккаунты с высокой конверсией "доминируют" над аккаунтами с низкой конверсией в плане количества подписчиков.

Высокой конверсией будем считать конверсию больше средней. Остальное - низкая. Посчитайте суммы подписчиков для каждой платформы, поделите одно на другое и выведите разницу между самым большим значением и самым маленьким, а также платформы, которые соотвутствуют этим значениям.

Используйте магическую команду `%%time`, чтобы замерить, сколько времени ушло на исполнение вашего pandas-скрипта.

In [27]:
conversion_average=df.groupby('Platform')['Conversion Rate'].mean()

In [28]:
conversion_average

,Conversion Rate
Platform,
Facebook,0.028878
Instagram,0.029179
LinkedIn,0.029197
TikTok,0.029290
Twitter,0.029602


In [14]:
conversion_average=df.groupby('Platform', as_index=False)['Conversion Rate'].mean()
conversion_average=conversion_average.rename(columns={'Conversion Rate':'Mean'})
df3=df.merge(conversion_average,on='Platform')
df3['Type of Conversion']=(df3['Conversion Rate']>df3['Mean'])
df4=df3.groupby(['Platform', 'Type of Conversion'])['Follower Count'].sum().unstack(fill_value=0)
df4['Ratio']=df4[True]/df4[False]
print(df4['Ratio'].max()-df4['Ratio'].min())
print(df4['Ratio'].idxmax(), df4['Ratio'].idxmin())
print(df4['Ratio'].max(), df4['Ratio'].min())# (◡‿◡✿)
%%time

0.14886262659546368
Twitter Instagram
1.013693864500984 0.8648312379055204


UsageError: Line magic function `%%time` not found.


In [33]:
%%time

UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


#### Задание 9

А теперь решите задание 8 чисто питоном. Никаких функций и методов pandas. Только питоновские циклы. Замерьте время выполнения кода. Наконец, сравните время в задании 8 и 9. Напишите ниже, кто же победил: чистый python и pandas?

**Hint**: Чтобы итерироваться по датафрейму, можно из него сделать генератор через [pd.DataFrame.iterrows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) или [pd.DataFrame.itertuples](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples). К слову, это не все способы итерироваться по датафрейму.

In [19]:
average_conversion_rate = df.groupby('Platform', as_index=False)['Conversion Rate'].mean()
average_conversion_rate = average_conversion_rate.rename(columns={'Conversion Rate': 'Mean'})
df = df.merge(average_conversion_rate, on='Platform')

high_conversion = {}
low_conversion = {}

for index, row in df.iterrows():
    platform = row['Platform']
    follower_count = row['Follower Count']

    if row['Conversion Rate'] > row['Mean']:
        if platform not in high_conversion:
            high_conversion[platform] = 0
        high_conversion[platform] += follower_count
    else:
        if platform not in low_conversion:
            low_conversion[platform] = 0
        low_conversion[platform] += follower_count

ratios = {}
for platform in set(high_conversion.keys()).union(set(low_conversion.keys())):
    high_count = high_conversion.get(platform, 0)
    low_count = low_conversion.get(platform, 1)
    ratios[platform] = high_count / low_count

max_ratio = max(ratios.values())
min_ratio = min(ratios.values())

max_platforms = [platform for platform, ratio in ratios.items() if ratio == max_ratio]
min_platforms = [platform for platform, ratio in ratios.items() if ratio == min_ratio]

print(max_ratio, max_platforms)
print(min_ratio, min_platforms)
print(max_ratio - min_ratio)
# (✿◠‿◠)

1.013693864500984 ['Twitter']
0.8648312379055204 ['Instagram']
0.14886262659546368


**А победителем является**: <А ТУТ МОЙ ОТВЕТ, Я ЗАМЕТИЛ, ЧТО В ЗАДАНИИ НУЖНО ЕЩЕ ЧТО-ТО НАПИСАТЬ ПОСЛЕ КОДА, ИНАЧЕ НЕ ПОЛУЧУ ПОЛНЫЙ БАЛЛ ЗА ЗАДАНИЕ>

#### Задание 10

Крайне серьезное задание. Отнеситесь к нему соответствующе. В ячейке ниже напишите ваш любимый анекдот или мем (только без баянов, окей?). Можно плохие. Помните, это задание на полный балл. Проверяющий работу ассистент должен улыбнуться.

Если вставляете картинку, то убедитесь, что вы ее не подгружаете локально. А то будет неудобно - потерять балл на этом задании, когда надо было выложить картинку на облако и прокинуть ссылку. И нет, нельзя сюда просто ссылку вставить. Либо ищите, как вставить картинку, либо смешной анекдот. Есть всего два стула - выбирайте...

In [ ]:
# ‿( ́ ̵ _-`)‿